# Compliance Data Work
<br>


In [ ]:
# Library Imports
import os
import import_ipynb
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from statistics import mean, median, mode, stdev 

# from Descript import extract
%matplotlib inline

In [ ]:
df = pd.read_csv("../DATA/2807.csv", sep=",", header=0)

# Convert Date attributes from type object to datetime object.
df['TreatmentStartDate'] = pd.to_datetime(df['TreatmentStartDate'], format='%Y-%m-%d %H:%M')
df['CtrlDatetime'] = pd.to_datetime(df['CtrlDatetime'], format='%Y-%m-%d %H:%M')
df['installDate'] = pd.to_datetime(df['installDate'], format='%Y-%m-%d')

# Set new attributes of Time and Date of Treatment
df['treatmentDate'] = df['TreatmentStartDate'].dt.date
# df['treatmentDate'] = pd.to_datetime(df['treatmentDate'])

df['treatmentTime'] = df['TreatmentStartDate'].dt.time
# df['treatmentTime'] = pd.to_datetime(df['treatmentTime'])

In [ ]:
# list(data) or 
len(df.columns)

In [ ]:
dfa = df[['CutitronicsClientID','CutitronicsSKUName','CutitronicsCartID', 'installDate', 'TreatmentStartDate', 'treatmentDate', 'treatmentTime', 'CartDispensedAmount','CartLevel', 'CartInitLevel', 'presrcibedAM', 'prescribedPM','PrescriptionType', 'CtrlSkinHealth']]

for i, row in dfa.iterrows():
    if row['treatmentTime'].hour < 4:
        am_val = 0
        pm_val = dfa.loc[i, 'CartDispensedAmount']
    elif row['treatmentTime'].hour > 17:
        am_val = 0
        pm_val = dfa.loc[i, 'CartDispensedAmount']
    else:
        am_val = dfa.loc[i, 'CartDispensedAmount']
        pm_val = 0
    dfa.loc[i,'actualAM'] = am_val
    dfa.loc[i,'actualPM'] = pm_val

dfa[['actualAM', 'actualPM', 'CtrlSkinHealth']] = dfa[['actualAM', 'actualPM', 'CtrlSkinHealth']].astype('int64')

dfa = dfa.groupby(['CutitronicsClientID', 'CutitronicsSKUName', 'treatmentDate']).agg({ 
    'installDate': 'last',
    'CutitronicsCartID': 'first',
    'CartDispensedAmount': 'sum',
    'CartLevel': 'last',
    'CartInitLevel': 'first',
    'presrcibedAM':'first',
    'prescribedPM':'last',
    'actualAM': 'sum',
    'actualPM':'sum',
    'PrescriptionType': 'first',
    'CtrlSkinHealth': 'mean'
})

for i, row in dfa.iterrows():
    if row['PrescriptionType'] == 'DAILY':
        if row['presrcibedAM'] == row['actualAM'] and row['prescribedPM'] == row['actualPM']:
            complied = 'Yes' # 1 = True (User Complied with prescription for that date)
        else:
            complied = 'No' # 0 = False (User did not comply with prescription for that date)
        dfa.loc[i,'compliance'] = complied
        
    else:
        if row['presrcibedAM'] == row['actualAM'] or row['prescribedPM'] == row['actualPM']:
            complied = 'Yes' # 1 = True (User Complied with prescription for that date)
        else:
            complied = 'No' # 0 = False (User did not comply with prescription for that date)
        dfa.loc[i,'compliance'] = complied
    
dfa[['actualAM', 'actualPM']] = dfa[['actualAM', 'actualPM']].astype('int64')

a = dfa['compliance'].value_counts(normalize=True) * 100
labels = dfa['compliance'].value_counts().index.tolist()
print(a)

plt.style.use('ggplot')

plt.bar(labels, a, color='green')
plt.xlabel(labels)
plt.ylabel("Percentage of Users")
plt.title("Percentage of Users Complying with Prescription")
plt.show()

asR = dfa.loc[dfa['PrescriptionType'] == 'AS REQUIRED'].reset_index()
df = dfa.reset_index()

In [ ]:
def missed(dfName):
    # Assign global to returned DFs
    global df
    global grouped_df
    
    # Assign df to the df name provided
    df = dfName
    
    # Apply funcion used for finding missed usage dates
    def foo(gr):
        gr = gr.set_index('treatmentDate')
        idx = pd.date_range(gr.index.min(), gr.index.max())
        gr.index = pd.DatetimeIndex(gr.index)
        gr = gr.reindex(idx, fill_value=0)
        return gr
    
    # Groupby and Apply function
    df = df[df['PrescriptionType'] == 'DAILY'].groupby(['CutitronicsClientID', 'CutitronicsSKUName']).apply(func=foo)
    
    # Groupy causes duplicate column with one as index
    # The col version is dropped to allow for reset of index
    df = df.drop(['CutitronicsClientID', 'CutitronicsSKUName'], axis = 1) 
    grouped_df = df
    df = df.reset_index()
    df =df.rename(columns={"level_2": "treatmentDate"})
    
    # Default of 0 is applied to all vlaues in missed usage row.
    # Alters comliance value to 'Missed' rather than 0
    for i, row in df.iterrows():
            if row['compliance'] == 0:
                complied = 'MISSED'
                date = row['treatmentDate']
            else:
                complied = row['compliance']
            df.loc[i,'compliance'] = complied
        
# Call function with df created previously
missed(df)

# # Join DF's back together
main_df = pd.concat([df, asR], join='inner').reindex()

In [ ]:
df.groupby(['CutitronicsClientID','CutitronicsSKUName'])
df.head()

In [ ]:
a = main_df.groupby(['CutitronicsClientID', 'CutitronicsSKUName'])
print(a.head())
for ID, data in main_df.groupby(['CutitronicsClientID', 'CutitronicsSKUName']):
    for i, row in data.iterrows():
        if i == 0:
            a = data.loc[i, 'CartInitLevel'] - data.loc[i, 'CartLevel']
            data.loc[i, 'productUsed'] = a
        else:
            a = data.loc[i-1, 'CartLevel'] - data.loc[i, 'CartLevel']
            data.loc[i, 'productUsed'] = a

In [ ]:
test = main_df.loc[main_df['CutitronicsClientID'] == 'client5']
test.head()
for i, row in test.iterrows():
            print(i)
            if i == 0:
                print("here")
                a = df.loc[i, 'CartInitLevel'] - df.loc[i, 'CartLevel']
                print(a)
            else:
                a = df.loc[i-1, 'CartLevel'] - df.loc[i, 'CartLevel']
                print(a)

In [ ]:
# for i, row in main_df.iterrows():
#     d1 = row['treatmentDate']
#     d0 = row['installDate']
    
#     delta = d1-d0
#     main_df.loc[i, 'daysInstalled'] = delta

# d1 = df['treatmentDate'][3]
# d0 = df['installDate'][3]
# delta = d1 - d0
# print(delta.days)

In [ ]:
main_df.head()

In [ ]:
values = df['compliance'].value_counts(normalize=True) * 100
labels = df['compliance'].value_counts().index.tolist()
print(values)

plt.style.use('ggplot')
plt.bar(labels, values)
plt.xlabel(labels)
plt.ylabel("Percentage of Users")
plt.title("Percentage of Users Complying with Prescription")
plt.show()

In [ ]:
df.groupby(['CutitronicsSKUName']).plot(x='treatmentDate', y='CartLevel', style='o')  
plt.title('Dispensed Amount vs CartLevel')  
plt.xlabel('Dispensed Amount')  
plt.ylabel('Cart Level')  
plt.show()

In [ ]:
for ID, data in main_df.groupby(['CutitronicsClientID', 'CutitronicsSKUName']):
    values = data['compliance'].value_counts(normalize=True) * 100
    labels = data['compliance'].value_counts().index.tolist()
    
    plt.pie(values, labels=labels,
        autopct='%5.0f%%', shadow=True)
    
    #draw a circle at the center of pie to make it look like a donut
    centre_circle = plt.Circle((0,0),0.75,color='black', fc='white',linewidth=1)
    fig = plt.gcf()
    fig.gca().add_artist(centre_circle)
    
    # Set aspect ratio to be equal so that pie is drawn as a circle.
    plt.axis('equal')
    plt.title('{} Compliance \n'.format(ID))
    plt.show()
    

In [ ]:
for ID, data in main_df.groupby(['CutitronicsClientID', 'CutitronicsSKUName']):
    data.index = data['treatmentDate']
    plt.title("Skin Health Rating for {}".format(ID))
    #plot
    plt.figure(figsize=(16,8))
    plt.plot(data['CtrlSkinHealth'], label='Skin Health history')

In [ ]:

values = main_df['compliance'].value_counts(normalize=True) * 100
labels = main_df['compliance'].value_counts().index.tolist()

colors = ['yellow', 'green', 'red']
explode = (0, 0, 0, 0)  # explode a slice if required

plt.pie(values, labels=labels,
        autopct='%1.1f%%', shadow=True)
        
#draw a circle at the center of pie to make it look like a donut
centre_circle = plt.Circle((0,0),0.75,color='black', fc='white',linewidth=1)
fig = plt.gcf()
fig.gca().add_artist(centre_circle)


# Set aspect ratio to be equal so that pie is drawn as a circle.
plt.axis('equal')
plt.title('Compliance to Prescription \n')
plt.show() 


In [ ]:

MP_KEY = 'pk.eyJ1IjoiYW5keWZhcnJlbGw3IiwiYSI6ImNrZDV6bmh2bTA2MzQyc241YXdoOWRkY2sifQ.Jt6edR8CCaS88tGae3m12A'
a = pd.read_csv("../DATA/2807.csv", sep=",", header=0)

In [ ]:

import plotly.express as px

# fig = px.scatter_mapbox(a, lat="clientLat", lon="clientLon", color="Area", hover_name='Area', hover_data=['CtrlSkinHealth'], zoom=4)
# fig.update_layout(mapbox_style="light", mapbox_accesstoken=MP_KEY)
# fig.show()


In [ ]:
import plotly.graph_objects as go

# values = main_df['compliance'].value_counts(normalize=True) * 100
# labels = main_df['compliance'].value_counts().index.tolist()

# print(values)

# # Use `hole` to create a donut-like pie chart
# fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.5)])
# fig.show()

In [ ]:
for ID, data in main_df.groupby(['CutitronicsClientID']):
    values = data['compliance'].value_counts(normalize=True) * 100
    labels = data['compliance'].value_counts().index.tolist()
    
    # Use `hole` to create a donut-like pie chart
    fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.6, pull=[0, 0, 0.1])])
    fig.update_layout(title="{}'s Compliance".format(ID))
    fig.show()

In [ ]:
def compPercent(series):
    a = series.value_counts()
    b = sum(a)
    if 'Yes' in a:
        return round(a['Yes'] / b * 100, 2)
    else:
        return 0;

In [ ]:

new_df = main_df.groupby(['CutitronicsClientID', 'treatmentDate']).agg({ 
    'CtrlSkinHealth': 'mean',
    'compliance': compPercent
})

new_df = new_df.reset_index()

fig = px.line(new_df, x="treatmentDate", y="CtrlSkinHealth", color='CutitronicsClientID', hover_data=['compliance'])
fig.show()


<br> <br>
# MACHINE LEARNING  <br>

In [ ]:
df3 = new_df.loc[new_df['CutitronicsClientID'] == 'client1'].groupby(['treatmentDate']).agg({ 
    'CtrlSkinHealth': 'mean'
})
df3 = df3.reset_index()

In [ ]:
rollingHealth = df3['CtrlSkinHealth'].rolling(5).mean()

fig = px.line(df3, x="treatmentDate", y=rollingHealth)
fig.show()

In [ ]:
df4 = df3.set_index('treatmentDate')

In [ ]:
# df4.loc['2020-06-19', 'CtrlSkinHealth'] = 74
# df4.loc['2020-06-20', 'CtrlSkinHealth'] = 76

In [ ]:
df4.plot(style='k.')
plt.show()

In [ ]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    #Determing rolling statistics
    rolmean = timeseries.rolling(7).mean()
    rolstd = timeseries.rolling(7).std()
    #Plot rolling statistics:
    plt.plot(timeseries, color='blue',label='Original')
    plt.plot(rolmean, color='red', label='Rolling Mean')
    plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean and Standard Deviation')
    plt.show(block=False)
    
    #perform dickey fuller test  
    print("Results of dickey fuller test")
    adft = adfuller(timeseries['CtrlSkinHealth'],autolag='AIC')
    # output for dft will give us without defining what the values are.
    #hence we manually write what values does it explains using a for loop
    output = pd.Series(adft[0:4],index=['Test Statistics','p-value','No. of lags used','Number of observations used'])
    for key,values in adft[4].items():
        output['critical value (%s)'%key] =  values
    print(output)
    
test_stationarity(df4)

In [ ]:
df_log = np.log(df4)
moving_avg = df_log.rolling(7).mean()
std_dev = df_log.rolling(7).std()
plt.plot(df_log)
plt.plot(moving_avg, color="red")
plt.plot(std_dev, color ="black")
plt.show()

df_log_moving_avg_diff = df_log-moving_avg
df_log_moving_avg_diff.dropna(inplace=True)

test_stationarity(df_log_moving_avg_diff)

weighted_average = df_log.ewm(halflife=7, min_periods=0,adjust=True).mean()

logScale_weightedMean = df_log-weighted_average
from pylab import rcParams
rcParams['figure.figsize'] = 10,6
logScale_weightedMean.dropna(inplace=True)
test_stationarity(logScale_weightedMean)



In [ ]:
from statsmodels.tsa.stattools import acf,pacf
# we use d value here(data_log_shift)
acf = acf(df_log, nlags=5)
pacf= pacf(df_log, nlags=5,method='ols')#plot PACF
plt.subplot(121)
plt.plot(acf) 
plt.axhline(y=0,linestyle='-',color='blue')
plt.axhline(y=-1.96/np.sqrt(len(logScale_weightedMean)),linestyle='--',color='black')
plt.axhline(y=1.96/np.sqrt(len(logScale_weightedMean)),linestyle='--',color='black')
plt.title('Auto corellation function')
plt.tight_layout()#plot ACF
plt.subplot(122)
plt.plot(pacf) 
plt.axhline(y=0,linestyle='-',color='blue')
plt.axhline(y=-1.96/np.sqrt(len(logScale_weightedMean)),linestyle='--',color='black')
plt.axhline(y=1.96/np.sqrt(len(logScale_weightedMean)),linestyle='--',color='black')
plt.title('Partially auto corellation function')
plt.tight_layout()

In [ ]:
df_log_diff = df_log - df_log.shift()
plt.title("Shifted timeseries")
plt.xlabel("Date")
plt.ylabel("Consumption")
plt.plot(df_log_diff)#Let us test the stationarity of our resultant series
df_log_diff.dropna(inplace=True)
test_stationarity(df_log_diff)

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
model = ARIMA(df_log, order=(7,1,5))
result_AR = model.fit(disp = 0)
plt.plot(df_log_diff)
plt.plot(result_AR.fittedvalues, color='green')
plt.title("sum of squares of residuals")
print('RSS : %f' %sum((result_AR.fittedvalues-df_log_diff["CtrlSkinHealth"])**2))

In [ ]:
result_AR.plot_predict(1,65)
x=result_AR.forecast(steps=200)

In [ ]:
# Import Statsmodels
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

In [ ]:
new_df.to_csv('finalDF.csv', index=False)

In [ ]:
# new_df.loc[new_df['CutitronicsClientId'] == 'client5']

df5 = new_df.loc[new_df['CutitronicsClientID'] == 'client1'].groupby(['treatmentDate']).agg({ 
# df5 = new_df.groupby(['treatmentDate']).agg({ 
    'CtrlSkinHealth': 'mean',
    'compliance': 'mean'
})
df5 = df5.reset_index()
df5 = df5.set_index('treatmentDate')


In [ ]:
df5 = df5[['CtrlSkinHealth','compliance']]
df5['CtrlSkinHealth'] = df5['CtrlSkinHealth'].astype('float32')

# Plot
fig, axes = plt.subplots(nrows=2, ncols=1, dpi=120, figsize=(10,6))
for i, ax in enumerate(axes.flatten()):
    data = df5[df5.columns[i]]
    ax.plot(data, color='red', linewidth=1)
    # Decorations
    ax.set_title(df5.columns[i])
    ax.xaxis.set_ticks_position('none')
    ax.yaxis.set_ticks_position('none')
    ax.spines["top"].set_alpha(0)
    ax.tick_params(labelsize=6)

plt.tight_layout();

from statsmodels.tsa.stattools import grangercausalitytests
maxlag=12
test = 'ssr_chi2test'
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
#     """Check Granger Causality of all possible combinations of the Time series.
#     The rows are the response variable, columns are predictors. The values in the table 
#     are the P-Values. P-Values lesser than the significance level (0.05), implies 
#     the Null Hypothesis that the coefficients of the corresponding past values is 
#     zero, that is, the X does not cause Y can be rejected.

#     data      : pandas dataframe containing the time series variables
#     variables : list containing names of the time series variables.
#     """
    df5 = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df5.columns:
        for r in df5.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df5.loc[r, c] = min_p_value
    df5.columns = [var + '_x' for var in variables]
    df5.index = [var + '_y' for var in variables]
    print("Granger Causality Results \n\n", df5)
    print("\n\n")

grangers_causation_matrix(df5, variables = df5.columns)

from statsmodels.tsa.vector_ar.vecm import coint_johansen

def cointegration_test(df5, alpha=0.05): 
#     """Perform Johanson's Cointegration Test and Report Summary"""
    out = coint_johansen(df5,-1,5)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-alpha)]]
    def adjust(val, length= 6): return str(val).ljust(length)

    # Summary
    print("Cointegration Test \n\n" + 'Name   ::  Test Stat > C(95%)    =>   Signif  \n', '--'*20)
    for col, trace, cvt in zip(df5.columns, traces, cvts):
        print(adjust(col), ':: ', adjust(round(trace,2), 9), ">", adjust(cvt, 8), ' =>  ' , trace > cvt)

cointegration_test(df5)


nobs = 10
df5_train, df5_test = df5[0:-nobs], df5[-nobs:]

# Check size
print("\n\n", df5_train.shape)  # (53, 2)
print(df5_test.shape, "\n\n")  # (4, 2)


def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")
        
# ADF Test on each column
for name, column in df5_train.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

# df_differenced = df5_train
# First Difference
df_differenced = df5_train.diff().dropna()
# ADF Test on each column of 1st Differences Dataframe
for name, column in df_differenced.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

# # Second Difference
# df_differenced = df_differenced.diff().dropna()
# # ADF Test on each column of 1st Differences Dataframe
# for name, column in df_differenced.iteritems():
#     adfuller_test(column, name=column.name)
#     print('\n')

In [ ]:

    
    
model = VAR(df_differenced)
for i in [1,2,3,4,5,6,7,8,9]:
    result = model.fit(i)
    print('Lag Order =', i)
    print('AIC : ', result.aic)
    print('BIC : ', result.bic)
    print('FPE : ', result.fpe)
    print('HQIC: ', result.hqic, '\n')

x = model.select_order(maxlags=12)
print(x.summary())

model_fitted = model.fit(8)
print(model_fitted.summary())

# Get the lag order
lag_order = model_fitted.k_ar
print(lag_order)  #> 4

# Input data for forecasting
forecast_input = df_differenced.values[-lag_order:]
print("forecast Input", forecast_input)

# Forecast
fc = model_fitted.forecast(y=forecast_input, steps=nobs)
df_forecast = pd.DataFrame(fc, index=df5.index[-nobs:], columns=df5.columns + '_2d')
df_forecast = df_forecast.astype('int32')
print(df_forecast.head(15))

def invert_transformation(df_train, df_forecast, second_diff=False):
    """Revert back the differencing to get the forecast to original scale."""
    df_fc = df_forecast.copy()
    columns = df5_train.columns
    for col in columns:        
        # Roll back 2nd Diff
        if second_diff:
            df_fc[str(col)+'_1d'] = (df5_train[col].iloc[-1]-df5_train[col].iloc[-2]) + df_fc[str(col)+'_2d'].cumsum()
       # Roll back 1st Diff
        df_fc[str(col)+'_forecast'] = df5_train[col].iloc[-1] + df_fc[str(col)+'_2d'].cumsum()
    return df_fc

df_results = invert_transformation(df5_train, df_forecast, second_diff=False)  
print(df_results[['CtrlSkinHealth_forecast']])
# df_results.loc[:, ['CtrlSkinHealth', 'compliance']]

# fig, axes = plt.subplots(nrows=2, ncols=1, dpi=120, figsize=(10,6))
# for i, (col,ax) in enumerate(zip(df5.columns, axes.flatten())):
#     df_forecast[col+'_2d'].plot(legend=True, ax=ax).autoscale(axis='x',tight=True)
#     df5_test[col][-nobs:].plot(legend=True, ax=ax);
#     ax.set_title(col + ": Forecast vs Actuals")
#     ax.xaxis.set_ticks_position('none')
#     ax.yaxis.set_ticks_position('none')
#     ax.spines["top"].set_alpha(0)
#     ax.tick_params(labelsize=6)

from statsmodels.stats.stattools import durbin_watson
out = durbin_watson(model_fitted.resid)

def adjust(val, length= 6): return str(val).ljust(length)

for col, val in zip(df5.columns, out):
    print("\n\n Durbin Watson Statistic:",adjust(col), ':', round(val, 2))

fig, axes = plt.subplots(nrows=2, ncols=1, dpi=150, figsize=(10,10))
for i, (col,ax) in enumerate(zip(df5.columns, axes.flatten())):
    df_results[col+'_forecast'].plot(legend=True, ax=ax).autoscale(axis='x',tight=True)
    df5_test[col][-nobs:].plot(legend=True, ax=ax);
    ax.set_title(col + ": Forecast vs Actuals")
    ax.xaxis.set_ticks_position('none')
    ax.yaxis.set_ticks_position('none')
    ax.spines["top"].set_alpha(0)
    ax.tick_params(labelsize=6)

plt.tight_layout();

In [ ]:
from statsmodels.tsa.stattools import acf
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    mins = np.amin(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    maxs = np.amax(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse, 'corr':corr, 'minmax':minmax})

print('Forecast Accuracy of: compliance')
accuracy_prod = forecast_accuracy(df_results['compliance_forecast'].values, df5_test['compliance'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,8))

print('\nForecast Accuracy of: CtrlSkinHealth')
accuracy_prod = forecast_accuracy(df_results['CtrlSkinHealth_forecast'].values, df5_test['CtrlSkinHealth'])
for k, v in accuracy_prod.items():
    print(adjust(k), ': ', round(v,8))

In [ ]:
# Plot
fig, axes = plt.subplots(nrows=2, ncols=1, dpi=120, figsize=(10,6))
for i, ax in enumerate(axes.flatten()):
    data = df5[df5.columns[i]]
    ax.plot(data, color='red', linewidth=1)
    # Decorations
    ax.set_title(df5.columns[i])
    ax.xaxis.set_ticks_position('none')
    ax.yaxis.set_ticks_position('none')
    ax.spines["top"].set_alpha(0)
    ax.tick_params(labelsize=6)

plt.tight_layout();

In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests
maxlag=10
test = 'ssr_chi2test'
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
#     """Check Granger Causality of all possible combinations of the Time series.
#     The rows are the response variable, columns are predictors. The values in the table 
#     are the P-Values. P-Values lesser than the significance level (0.05), implies 
#     the Null Hypothesis that the coefficients of the corresponding past values is 
#     zero, that is, the X does not cause Y can be rejected.

#     data      : pandas dataframe containing the time series variables
#     variables : list containing names of the time series variables.
#     """
    df5 = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df5.columns:
        for r in df5.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df5.loc[r, c] = min_p_value
    df5.columns = [var + '_x' for var in variables]
    df5.index = [var + '_y' for var in variables]
    return df5

grangers_causation_matrix(df5, variables = df5.columns)  

In [ ]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

def cointegration_test(df5, alpha=0.05): 
    """Perform Johanson's Cointegration Test and Report Summary"""
    out = coint_johansen(df5,-1,5)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-alpha)]]
    def adjust(val, length= 6): return str(val).ljust(length)

    # Summary
    print('Name   ::  Test Stat > C(95%)    =>   Signif  \n', '--'*20)
    for col, trace, cvt in zip(df5.columns, traces, cvts):
        print(adjust(col), ':: ', adjust(round(trace,2), 9), ">", adjust(cvt, 8), ' =>  ' , trace > cvt)

cointegration_test(df5)

In [ ]:
nobs = 5
df5_train, df5_test = df5[0:-nobs], df5[-nobs:]

# Check size
print(df5_train.shape)  # (53, 2)
print(df5_test.shape)  # (4, 2)

In [ ]:
def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.") 

In [ ]:
# ADF Test on each column
for name, column in df5_train.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

In [ ]:
model = VAR(df5_train)
for i in [1,2,3,4,5,6,7,8,9]:
    result = model.fit(i)
    print('Lag Order =', i)
    print('AIC : ', result.aic)
    print('BIC : ', result.bic)
    print('FPE : ', result.fpe)
    print('HQIC: ', result.hqic, '\n')

In [ ]:
x = model.select_order(maxlags=9)
x.summary()

In [ ]:
model_fitted = model.fit(8)
model_fitted.summary()

In [ ]:
# Get the lag order
lag_order = model_fitted.k_ar
print(lag_order)  #> 4

# Input data for forecasting
forecast_input = df5_train.values[-lag_order:]
forecast_input

# Forecast
fc = model_fitted.forecast(y=forecast_input, steps=nobs)
df_forecast = pd.DataFrame(fc, index=df5.index[-nobs:], columns=df5.columns + '_2d')
df_forecast = df_forecast.astype('int32')

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, dpi=120, figsize=(10,6))
for i, (col,ax) in enumerate(zip(df5.columns, axes.flatten())):
    df_forecast[col+'_2d'].plot(legend=True, ax=ax).autoscale(axis='x',tight=True)
    df5_test[col][-nobs:].plot(legend=True, ax=ax);
    ax.set_title(col + ": Forecast vs Actuals")
    ax.xaxis.set_ticks_position('none')
    ax.yaxis.set_ticks_position('none')
    ax.spines["top"].set_alpha(0)
    ax.tick_params(labelsize=6)

In [ ]:
# # Let's take a look at my dataframe

# fig, ax = plt.subplots(df5.shape[1], 1, figsize=(10, 10))
# plt.subplots_adjust(hspace=0.3)

# for i in range(df5.shape[1]):
#     color = 'r' if i == 0 else 'b'
#     ax[i].plot(df[ df5.columns[i] ], color, label = df5.columns[i])
#     ax[i].legend(prop={'size': 8})
# plt.show()

In [ ]:

#1 Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# #2 Importing the dataset
X = df5.iloc[:,0:1].values.astype(float)
y = df5.iloc[:,1:2].values.astype(float)


#3 Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()
X = sc_X.fit_transform(X)
y = sc_y.fit_transform(y)


#4 Fitting the Support Vector Regression Model to the dataset
# Create your support vector regressor here
from sklearn.svm import SVR
# most important SVR parameter is Kernel type. It can be #linear,polynomial or gaussian SVR. We have a non-linear condition #so we can select polynomial or gaussian but here we select RBF(a #gaussian type) kernel.
regressor = SVR(kernel='linear')
regressor.fit(X,y)



In [ ]:

svr = SVR(kernel='linear').fit(X, y)
print(svr)

In [ ]:
yfit = svr.predict(X)

In [ ]:
plt.scatter(X, y, s=5, color="blue", label="original")
plt.plot(X, yfit, lw=2, color="red", label="fitted")
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
score = svr.score(X,y)
print("R-squared:", score)
print("MSE:", mean_squared_error(y, yfit))

In [ ]:
df5.head()

In [ ]:
X = df5.index
X = pd.to_numeric(X)
X